# CART 树和剪枝
由于找到的数据集为连续数据集, 并且数据较多, 这里只构建 CART 树并基于 CART 树进行剪枝

## CART 树

---

1. 核心特征
    - 使用基尼指数用来划分数据集
    - 采取二分法构建树
1. 相对 ID3 和 C4.5 树的特征

    |            | ID3    | C4.5     | CART     |
    |------------|--------|----------|----------|
    | 核心特征   | 信息熵 | 信息增益 | 基尼系数 |
    | 能否剪枝   | 不能   | 能       | 能       |
    | 连续值处理 | 不能   | 能       | 能       |
    | 缺失值处理 | 不能   | 不能     | 能       |

    - 可以处理连续数据和缺失数据
    - 二分法构建树, 构建速度比较快
    - 拟合优度更佳
    - 不停二分离散特征, 这让已经被选择过的标签可以继续参与选择
1. 基尼指数(Gini Index)
    - 基尼指数表示样本的 "**不纯度**", 基尼指数越大, 样本越不纯
    - 整体基尼指数的计算, 公式为$$Gini(D)=1-\sum_{i=1}^{n} p(x_{i})^{2}$$
    其中 $p(x_{i})$ 为分类 $i$ 出现的次数
    - 条件基尼指数的计算$$Gini(D|A=a)=\frac{|D|}{|D_{1}|}Gini(D_{1})+\frac{|D|}{|D_{2}|}Gini(D_{2})$$
    其中 $D_1$ 和 $D_2$ 分别是当特征 $A=a$ 时对数据集的划分. 当特征 $A$ 为连续型时, 要进行连续变量离散化处理, 也就是说, 应该使用 $A\geq a$ (或 $A\leq a$) 和 $A<a$ (或 $A> a$) 来划分数据集.

## 剪枝

---

### 前剪枝(pre-pruning)策略
1. 设定深度阈值, 深度达到阈值后停止树的生长
1. 定义实例数阈值，当达到某个结点的实例个数小于该阈值时就可以停止树的生长
### 后剪枝(post-pruning)策略
比较常用的策略一共有 3 种, 分别是 REP, PEP, 和CCP. 本例仅介绍和使用 CCP.

### CCP(代价复杂度剪枝)介绍
该算法为子树 $T_t$ 定义了代价 (cost) 和复杂度 (complexity), 以及一个可由用户设置的衡量代价与复杂度之间关系的参数 $\alpha$. 其中, 代价指在剪枝过程中因子树 $T_t$ 被叶节点替代而增加的错分样本, 复杂度表示剪枝后子树 $T_t$ 减少的叶结点数, 表面误差增益率 $\alpha$ 表示剪掉某节点的左右子树后树的复杂度降低程度与代价间的关系, 定义一个非叶子节点上的 $\alpha$ 值为
$$
\alpha=\frac{R(t)-R(T_{t})}{|N_{1}|-1}
$$
其中:
- $|N_1|$ 为子树中的节点个数
- $R(t)$ 为结点 $t$ 的错误代价, 计算公式为 $R(t)=r(t)\cdot p(t)=m/n$, $r(t)$ 为结点 $t$ 的错分样本率, $p(t)$为落入结点 $t$ 的样本占所有样本的比例, $m$ 为节点 $t$ 的分类错误的样本数, $n$ 为总的样本数.
- $R(T_{t})$：子树 $T_t$ 错误代价，计算公式为 $R(T_{t})=\sum R(i)$, $i$ 为子树 $T_t$ 的叶节点.

对于完全决策树的每个非叶子节点, 计算它们的 $\alpha$ 值, 然后循环剪掉 $\alpha$ 最小的子树, 直到只剩下根节点. 每一步都可得到一个剪枝树, 由此得到一个剪枝树序列 $\{T_{0},\ T_{1},\ ...,\ T{m}\}$, 其中 $T_0$ 是完全决策树, $T_m$ 是根节点. 

然后使用 1-SE(1 standard error of minimum error) 规则从上述序列中挑选一个最优子树. 

1-SE 规则: 假定一个验证样本集含有 $N'$ 个样本, 使用使用上述剪枝树对样本集分类, 记 $T_i$ 上被错分的样本数为 $E_i$ 并令 $E'$ 为序列 $\{E_i \}$ 的最小值. 计算标准样本误
$$
SE(E')=\sqrt{\frac{E'(N'-E')}{N'}}
$$

找到满足 $E_i\leq E'+SE(E')$ 且节点数最少的剪枝树即为所求最优剪枝树 $T_i$.

## 程序实现

---

### 创建数据
BaseData 中第一行为标签数据, 前 3/4 为样本集, 后 1/4 为验证集, happy = 1 为 True, 否则为 False

In [4]:
import copy
from pandas import read_csv as read
from math import sqrt
def DataCreate():
    BaseData=read('./assets/data.csv',header=0,converters={'happy':lambda x:bool(int(x))})
    #从 data.csv 读取数据, 第一列为索引列, 第一行为标题行, converters 表示对 satisfaction_level 列做改动
    length=len(BaseData)
    Dataset=BaseData.iloc[0:3*length//4] #取前3/4为样本集
    VarifySet=BaseData.iloc[3*length//4:length] #取后1/4为验证集
    return Dataset, VarifySet

### 计算基尼指数
这里计算的是非条件的基尼指数, 条件基尼指数的计算只要切分数据集后用非条件的基尼指数进行四则运算即可

In [5]:
def Gini(Dataset):
    KindIndex=Dataset.columns[-1] #取最后一列的列标题, 也就是特征的标签
    KindNum=dict.fromkeys(set(Dataset[KindIndex].tolist()),0) #取最后一列的数据,并去除重复转换为字典, 默认值为 0
    for index, row in Dataset.iterrows(): #iterrows方法, 返回一个生成器, 第一项为数据索引, 第二项为数据
        KindNum[row[-1]] += 1 #计算每种分类的数量
    GiniIndex = 1 #计算sum(p(xi)**2)
    for example in KindNum.keys():
        GiniIndex -= (KindNum[example]/len(Dataset))**2
    return GiniIndex #返回基尼指数

### 数据切分
即求公式中的 $D_1$ 和 $D_2$, 切分点为 label 中的 value 值. 

只认为 string 是非连续的, 其它类型的切分都使用数值切分法

In [6]:
def SetSpilt(Dataset, label, value):
    if type(value) == str : #如果类型属于字符串,则执行离散切分
        subset1 = Dataset[Dataset[label] == value] #使用了dataframe的布尔索引功能, 筛选 label 列中等于 value 的数据, 下同
        subset2 = Dataset[Dataset[label] != value]
    else: #如果不是字符串, 则执行离散化切分
        subset1=Dataset[Dataset[label] >= value]
        subset2=Dataset[Dataset[label] < value]
    return subset1, subset2 #返回值为切分后的两个数据集
# Dataset, VarifySet=DataCreate()
# set1,set2=SetSpilt(Dataset,'package','a')

### 寻找最佳切分点
- 比较所有标签切分点的基尼指数, 然后取基尼指数最小的特征进行切分
- 对于连续数据切分点的选择, 把可能出现的取值排序, 然后取中位数, 利用中位数切分, 这就是连续数据离散化的思想. 但是这里我偷了个懒，直接令切分点等于数据集中的各特征值.

In [7]:
def BestSpilt(Dataset):
    bestGini = 1 #初始化最优Gini指数为1
    modifier = 1 / len(Dataset) #计算1/|D|
    BestSpiltPoint=() #最佳切分点, 是一个元组
    for label in Dataset.columns[:-2]: #从所有特征中循环
        FeatureSet = set(Dataset[label].tolist()) #特征值的集合
        if (len(FeatureSet) == 1): #如果该特征下只有一个特征值, 则已经被使用过, 跳过
            continue
        for feature in FeatureSet:
            subSet1,subSet2 = SetSpilt(Dataset, label, feature) #划分数据集
            if (len(subSet1) == 0)or(len(subSet2) == 0):
                continue
            conditionGini = modifier * (Gini(subSet1)/len(subSet1) + Gini(subSet2)/len(subSet2)) #计算条件gini指数
            if conditionGini < bestGini: #寻找最小gini指数
                bestGini = conditionGini
                BestSpiltPoint=((label,feature),subSet1,subSet2) #更新返回值
    if len(BestSpiltPoint)==0: #当前样本集在所有特征上的所有特征值都相同
        return 0
    return BestSpiltPoint

### 寻找数据集中最多的分类

In [8]:
def mostFeature(Dataset):
    KindIndex=Dataset.columns[-1]
    KindNum=dict.fromkeys(set(Dataset[KindIndex].tolist()),0) #取最后一列的数据,并去除重复转换为字典
    for index, row in Dataset.iterrows(): #iterrows方法, 返回一个生成器, 第一项为数据索引, 第二项为数据, 可用于循环
        KindNum[row[-1]] += 1 #计算每种分类的数量
    return max(KindNum,key=KindNum.get) #返回最大值

### 创建树的过程
本环节已经包含前剪枝

In [11]:
def CreateMytreeNoPrune(Dataset):
    #递归出口, 如果当前节点包含样本在所有特征上的特征值都相同,则终止递归, 返回数据集中最多的特征
    if BestSpilt(Dataset) == 0:
        return mostFeature(Dataset)
    
    ((label,feature),subSet1,subSet2)=BestSpilt(Dataset)

    myTree={'label':(label, feature), 'left':{}, 'right':{}}
    myTree['left']=CreateMytreeNoPrune(subSet1)
    myTree['right']=CreateMytreeNoPrune(subSet2)
    return myTree
dateset, VarifySet = DataCreate()
tree = CreateMytreeNoPrune(dateset)
tree

{'label': ('infoavail', 3),
 'left': {'label': ('policetrust', 2),
  'left': {'label': ('streetquality', 2),
   'left': {'label': ('infoavail', 5),
    'left': {'label': ('schoolquality', 2),
     'left': {'label': ('policetrust', 5),
      'left': {'label': ('schoolquality', 3),
       'left': {'label': ('housecost', 2),
        'left': {'label': ('schoolquality', 4),
         'left': {'label': ('housecost', 5),
          'left': True,
          'right': {'label': ('housecost', 4),
           'left': True,
           'right': {'label': ('schoolquality', 5),
            'left': {'label': ('housecost', 3), 'left': True, 'right': True},
            'right': {'label': ('housecost', 3),
             'left': False,
             'right': {'label': ('streetquality', 5),
              'left': True,
              'right': True}}}}},
         'right': {'label': ('housecost', 5), 'left': True, 'right': True}},
        'right': True},
       'right': True},
      'right': {'label': ('schoolquality

In [ ]:
def CreateMytree(Dataset, depth=0):
    depth += 1

    #递归出口1, 如果决策树的深度大于8层, 则停止递归,返回数据集中最多的特征
    if depth >= 8:
        return [mostFeature(Dataset), 0, 0]
    thrval = 0.05
    leastNum = 5

    #递归出口2, 如果当前节点包含的样本数小于leastNum, 或gini指数小于阈值thrval, 终止递归, 返回数据集中最多的特征
    if (len(Dataset)<leastNum) or (Gini(Dataset) < thrval):
        return [mostFeature(Dataset), 0, 0]
    
    #递归出口3, 如果当前节点包含样本在所有特征上的特征值都相同, 而样本仍然没有分纯(因为通过了递归出口2)则终止递归, 返回数据集中最多的特征
    if BestSpilt(Dataset) == 0:
        return [mostFeature(Dataset), 0, 0]
    
    ((label,feature),subSet1,subSet2)=BestSpilt(Dataset)
    ''''
    这是树的一个节点, 
    count表示验证集落入该节点的数量, 第一项为落入该节点的样本总数, 第二项为判断错误的样本数;
    AlphaCounted表示遍历时, 该节点有没有被遍历过;
    label表示进入左子树的条件, 样本的label值必须大于等于feature;
    left表示左节点, right表示右节点.
    '''
    myTree={'count':[0, 0],'AlphaCounted':False, 'label':(label, feature), 'left':{}, 'right':{}}
    myTree['left']=CreateMytree(subSet1, depth)
    myTree['right']=CreateMytree(subSet2,depth)
    return myTree


dateset, VarifySet = DataCreate()
tree = CreateMytree(dateset)
tree

{'count': [0, 0],
 'AlphaCounted': False,
 'label': ('infoavail', 3),
 'left': {'count': [0, 0],
  'AlphaCounted': False,
  'label': ('policetrust', 2),
  'left': {'count': [0, 0],
   'AlphaCounted': False,
   'label': ('streetquality', 2),
   'left': {'count': [0, 0],
    'AlphaCounted': False,
    'label': ('infoavail', 5),
    'left': {'count': [0, 0],
     'AlphaCounted': False,
     'label': ('schoolquality', 2),
     'left': {'count': [0, 0],
      'AlphaCounted': False,
      'label': ('policetrust', 5),
      'left': {'count': [0, 0],
       'AlphaCounted': False,
       'label': ('schoolquality', 3),
       'left': [True, 0, 0],
       'right': [True, 0, 0]},
      'right': {'count': [0, 0],
       'AlphaCounted': False,
       'label': ('schoolquality', 5),
       'left': [False, 0, 0],
       'right': [True, 0, 0]}},
     'right': [False, 0, 0]},
    'right': {'count': [0, 0],
     'AlphaCounted': False,
     'label': ('policetrust', 3),
     'left': {'count': [0, 0],
      

### 计算树中的叶节点个数 $|N_1|$

In [ ]:
def CountLeaf(Tree):
    sum = 0
    #递归计算树的叶节点, 如果子节点还不是叶节点, 则计算子树的叶节点个数, 如果是叶节点, 则返回
    for value in Tree.values():
        if type(value) == dict:
            sum += CountLeaf(value)
        elif type(value) == list and len(value) == 3:
            sum += 1
    return sum

### 利用所得决策树对单条样本进行分类

In [ ]:
def SinglePredict(example, Tree):
    tree = Tree
    path=[]
    while True:
        #如果递归应当结束, 则tree将不再是字典, 索引将抛出错误, 表明递归结束
        try:
            label = tree['label'][0]
            value = tree['label'][1]
            path.append(tree)
        except:
        #路径的计数列表第一项加一, 且如果该数据没有被正确分类, 则计数列表的第二项加一
            tree[1] += 1
            for value in path:
                value['count'][0] += 1
            if example.values[-1] != tree[0]:
                tree[2] += 1
                for value in path:
                    value['count'][1] += 1
            return Tree
        
        if type(value) == str:
            if example[label] == value: #如果相等则落入左子树, 不相等则落入右子树
                tree = tree['left']
            else:
                tree = tree['right']
        else:
            if example[label] >= value:
                tree = tree['left']
            else:
                tree = tree['right']

### 对于所有数据进行预测并形成完全剪枝树

In [ ]:
def Predict(VarifySet, Tree):
    for index, row in VarifySet.iterrows():
        SinglePredict(row, Tree)
    return Tree

### 计算表面误差增益率
计算的是树的根节点的 $\alpha$, 它的计算是由上自下的

In [ ]:
def CountR(Tree, leng):
    sum = 0
    #递归计算树的叶节点, 如果子节点还不是叶节点, 则计算子树的叶节点, 如果是叶节点, 则返回
    for value in Tree.values():
        if type(value) == dict:
            sum += CountR(value)
        elif (type(value) == list) and len(value) == 3:
            sum += value[2] / leng
    return sum

def alpha(Tree, leng):
    if Tree['count'][0] == 0:
        return 0
    RT = Tree['count'][1] / Tree['count'][0]
    N = CountLeaf(Tree)
    r = CountR(Tree, leng)
    return (RT - r) / (N - 1)

### 进行一次后剪枝
找到 $\alpha$ 值最小的内部节点, 对它进行剪枝, 并令其等于分到此处的样本的最多的那一类, 返回值为剪枝树.

In [ ]:
def singlePostPruning(Tree, VarifySet):
    leng = len(VarifySet)
    alphamin = 1.0
    tree = Tree
    NodeToPrune = ()
    path = []
    '''
    后序遍历树, 并计算alpha值
    后序遍历, 即先访问左子树, 后访问右子树, 最后访问根节点
    '''
    while Tree['AlphaCounted'] == False:
        path = []
        if type(tree['left']) == dict:#先尝试访问左子树
            if tree['left']['AlphaCounted'] == False:
                tree = tree['left']
                path.append('left')
                continue
        else:#如果树的左子树已经是叶子, 证明应该返回根节点重新递归
            tree['AlphaCounted'] = True
            if alpha(tree, leng) <= alphamin:
                alphamin = alpha(tree, leng)
                NodeToPrune = path
            tree = Tree
            continue
        
        if type(tree['right']) == dict:
            if tree['right']['AlphaCounted'] == False:
                tree = tree['right']
                path.append('right')
                continue
        else:#如果树的右子树已经是叶子, 则索引将抛出TypeError, 证明应该返回根节点重新递归
            tree['AlphaCounted'] = True
            if alpha(tree, leng) <= alphamin:
                alphamin = alpha(tree, leng)
                NodeToPrune = path
            tree = Tree
            continue
        #如果左右节点都已经计算过了, 则计算根节点的alpha值
        tree['AlphaCounted'] = True
        if alpha(tree, leng) <= alphamin:
            alphamin = alpha(tree, leng)
            NodeToPrune = path
        tree = Tree
    PrunedVarifySet = VarifySet.__deepcopy__()
    tree = Tree
    for item in NodeToPrune[:-1]:
        if item == 'left':
            PrunedVarifySet = PrunedVarifySet[PrunedVarifySet[tree['label'][0]]>=tree['label'][1]]
        else:
            PrunedVarifySet = PrunedVarifySet[PrunedVarifySet[tree['label'][0]]<tree['label'][1]]
        tree = tree[item]
    print(NodeToPrune[1])

    Tree = Predict(VarifySet, Tree)

    #下面在使得AlphaCounted值变回False
    while Tree['AlphaCounted'] == True:
        if type(tree['left']) == dict:#先尝试访问左子树
            if tree['left']['AlphaCounted'] == True:
                tree = tree['left']
                continue
        else:#如果树的左子树已经是叶子, 应该返回根节点重新递归
            tree['AlphaCounted'] = False
            tree = Tree
            continue
        
        if type(tree['right']) == dict:
            if tree['right']['AlphaCounted'] == True:
                tree = tree['right']
                continue
        else:#如果树的右子树已经是叶子, 应该返回根节点重新递归
            tree['AlphaCounted'] = False
            tree = Tree
            continue
        #如果左右节点都已经计算过了, 则计算根节点的alpha值
        tree['AlphaCounted'] = False
        tree = Tree
    return Tree
singlePostPruning(tree, VarifySet)

IndexError: list index out of range

### 求得剪枝树序列

In [ ]:
def PostPruning(VarifySet, Tree):
    PrunedTreeArray = []
    # try:
        # while type(Tree['left']) == dict and type(Tree['right']) == dict:
    PrunedTreeArray.append(singlePostPruning(Tree, VarifySet))
    # except:
    #     return PrunedTreeArray

### 使用 1-SE 规则挑选最佳剪枝树

In [ ]:
def SE(PrunedTreeArray, VarifySet):
    #min()函数的key参数用法, 使用key()函数的返回值来进行排序
    E = min(PrunedTreeArray, key=lambda x: x['count'][1])['count'][1]
    N = len(VarifySet)
    se = sqrt(E * (N - E) / N)
    #推导式创建列表
    goodTree = [Tree for Tree in PrunedTreeArray if Tree['count'][1] <= E + se]
    return min(goodTree, key=CountLeaf)

In [ ]:
SE(PostPruning(tree, VarifySet),VarifySet)